In [23]:
import numpy as np
import pandas as pd
import datetime
import math
import random
from datetime import datetime
from statistics import mean

In [24]:
x = [
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1],
    #A partir de aqui es relleno para el mini-batch
    [1,2],
    [1,3],
    [2,3],
    [2,4],
    [3,2],
    [3,5],
    [4,1]
]
y = [1.03,-1.44,4.53,2.24,13.27,5.62,21.53,#A partir de aqui es relleno para el mini-batch
     1.03,-1.44,4.53,2.24,13.27,5.62,21.53]

In [25]:
def signo(x):
  if x>=0:
    return 1
  else:
    return -1

In [26]:
def predict(modelo,x):
  if type(x[0]) == list:
    pred_v = []
    for x_it in x:
      pred = modelo[len(modelo)-1]
      for i in range(len(modelo)-1):
        pred+= modelo[i]*x_it[i]
      pred_v.append(pred)
    return pred_v
  else:
    pred = modelo[len(modelo)-1]
    for i in range(len(modelo)-1):
      pred+= modelo[i]*x[i]

    return pred

def MAE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  abs_v = [abs(ele) for ele in errs]

  return mean(abs_v)
def MSE(modelo,x,y):
  pred = predict(modelo,x)
  errs =[e1 - e2 for e1, e2 in zip(y,pred)]
  squ_v = [(ele)**2 for ele in errs]

  return mean(squ_v)

def MSE_GRAD(modelo,x,errs,w_it,l_rate,reg_term,Lasso,Ridge):
  if w_it < len(modelo)-1:
        #len(modelo)-1 porque el último peso es el sesgo y se ajusta
        #de forma distinta
        #Aqui se ajustan todos los pesos menos el bias

        #Cálculo del Σ((Ŷ-Y)*Xji)
        for er in range(len(errs)):
          errs[er]= errs[er]*x[er][w_it]
        if Lasso:
          modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)-reg_term*signo(modelo[w_it])
        elif Ridge:
          modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)-2*reg_term*modelo[w_it]
        else:
          modelo[w_it] = modelo[w_it]-1*(2*l_rate)*mean(errs)

  else:
    #Aqui se ajusta el bias
    modelo[w_it] = modelo[w_it] -1*(2*l_rate*mean(errs) )

def MAE_GRAD(modelo,x,errs,w_it,l_rate,reg_term,Lasso,Ridge):
  if w_it < len(modelo)-1:
        for er in range(len(errs)):
          errs[er]= signo(errs[er])*x[er][w_it]
        if Lasso:
          modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)-reg_term*signo(modelo[w_it])
        elif Ridge:
          modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)-2*reg_term*modelo[w_it]
        else:
          modelo[w_it] = modelo[w_it]-1*(l_rate)*mean(errs)
        #Aqui se ajusta el bias
  else:
    errs = [signo(x) for x in errs]
    modelo[w_it] = modelo[w_it] -1*(l_rate*mean(errs) )

In [27]:
predict([1,1,1],x)

[4, 5, 6, 7, 6, 9, 6, 4, 5, 6, 7, 6, 9, 6]

In [28]:
modelo = [0,0,0]
#[x1,x2,b]

In [29]:
def generate_mini_batch(x, y, batch_percentage):
  tam_mini_batch = math.trunc(len(x)*batch_percentage)
  mini_batch_x = []
  mini_batch_y = []
  x_copy = x.copy()
  y_copy = y.copy()

  while len(x_copy)>0:
    it_mini_batchx = []
    it_mini_batchy = []
    while len(it_mini_batchx)<tam_mini_batch:
      index = random.randint(0,len(x_copy)-1)
      it_mini_batchx.append(x_copy[index])
      it_mini_batchy.append(y_copy[index])
      x_copy.pop(index)
      y_copy.pop(index)
    mini_batch_x.append(it_mini_batchx)
    mini_batch_y.append(it_mini_batchy)

  return mini_batch_x,mini_batch_y


In [30]:
def linear_regression(modelo,x,y,n_it,l_rate,reg_term,func_ajuste, Lasso, Ridge, minibatch_mode, batch_percentage=None):
  for _ in range(n_it):
    if not minibatch_mode:
      errs = []
      pred = predict(modelo,x)
      errs = [e1 - e2 for e1, e2 in zip(pred,y)]

      for w_it in range(len(modelo)):
        errs_it = errs.copy()
        func_ajuste(modelo,x,errs_it,w_it,l_rate,reg_term,Lasso,Ridge)
    else:
      batch_x, batch_y = generate_mini_batch(x, y, batch_percentage)
      for a, b in zip(batch_x, batch_y):
        errs = []
        pred = predict(modelo,a)
        errs = [e1 - e2 for e1, e2 in zip(pred,b)]

        for w_it in range(len(modelo)):
          errs_it = errs.copy()
          func_ajuste(modelo,a,errs_it,w_it,l_rate,reg_term,Lasso,Ridge)
  return modelo

In [31]:
# Ajuste usando batch

print("Ajuste usando batch")

modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,0,MSE_GRAD,False,False, False)
print('Error final -> ',MSE(modelo,x,y))
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001, 0, MAE_GRAD, False, False, False)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)

# Ajuste usando minibatch

print("\nAjuste usando minibatch")
modelo = [1,1,1]
print('Ajuste con MSE')
print('Error inicial -> ',MSE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,0,MSE_GRAD,False,False, True, 0.2)
print('Error final -> ',MSE(modelo,x,y))
print(modelo)

modelo = [1,1,1]
print('\nAjuste con MAE')
print('Error inicial -> ',MAE(modelo,x,y))
linear_regression(modelo,x,y,20000,0.001,0,MAE_GRAD,False,False, True, 0.2)
print('Error final -> ',MAE(modelo,x,y))
print(modelo)

Ajuste usando batch
Ajuste con MSE
Error inicial ->  54.367314285714286


Error final ->  0.008417023656379178
[6.007147684087246, -2.489069952946704, 0.06976974256473045]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.05959183673453323
[6.008285714285928, -2.4745714285724416, -0.02685714285709419]

Ajuste usando minibatch
Ajuste con MSE
Error inicial ->  54.367314285714286
Error final ->  0.007876925065686236
[6.024089570888959, -2.4749187254978238, -0.015320358354594787]

Ajuste con MAE
Error inicial ->  5.974285714285714
Error final ->  0.06057142857129029
[6.007000000000242, -2.471999999999958, -0.0320000000000009]


Transformación del dataset a arrays para que el regresor pueda trabajar con él

In [47]:
data = pd.read_csv('diabetes.tab.csv',delimiter='\t')
def from_dataset_to_arrays(data):
  Y = data['Y']
  y = list(Y.values)
  data = data.drop(columns='Y')
  x = list(data.values)
  for a in range(len(x)):
    b = list(x[a])
    x[a] = b

  return x,y
x,y = from_dataset_to_arrays(data)

Nota, aquí si no se utiliza un learning rate pequeño no minimiza bien el error

In [49]:
modelo = []
for i in range(len(x[0])):
  modelo.append(1)
# print('Ajuste con MSE')
# print('Error inicial -> ',MAE(modelo,x,y))
# modelo =linear_regression(modelo,x,y,1000,0.00001,0,MSE_GRAD,False,False,False)
# print('Error final -> ',MAE(modelo,x,y))
# print(modelo)

In [51]:
def lasso_error(modelo,x,y,reg_term,error_func):
    
    reg_error = error_func(modelo,x,y)
    L1=0
    for weight in range(len(modelo)-1):
        L1 += abs(modelo[weight])
        
    return reg_error + L1*reg_term

def ridge_error(modelo,x,y,reg_term,error_func):
    
    reg_error = error_func(modelo,x,y)
    L2=0
    for weight in range(len(modelo)-1):
        L2 += (modelo[weight])**2

    return reg_error + L2*reg_term

In [54]:
print('Ajuste lasso y ridge con conjunto de datos diabetes')

reg_term = 0.01

print('\n---Ajuste lasso----')

modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('Ajuste con MSE')
print('Error inicial -> ',lasso_error(modelo,x,y,reg_term,MSE))
linear_regression(modelo,x,y,2000,0.000001,reg_term, MSE_GRAD, True, False, False)
print('Error final -> ',lasso_error(modelo,x,y,reg_term,MSE))
print(modelo)


modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('\nAjuste con MAE')
print('Error inicial -> ',lasso_error(modelo,x,y,reg_term,MAE))
linear_regression(modelo,x,y,2000,0.0001,reg_term,MAE_GRAD,True,False, False)
print('Error final -> ',lasso_error(modelo,x,y,reg_term,MAE))
print(modelo)

print('\n---Ajuste ridge:---')

modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('Ajuste con MSE')
print('Error inicial -> ',ridge_error(modelo,x,y,reg_term,MSE))
linear_regression(modelo,x,y,2000,0.000001,reg_term, MSE_GRAD,False, True, False)
print('Error final -> ',ridge_error(modelo,x,y,reg_term,MSE))
print(modelo)


modelo = []
for i in range(len(x[0])):
  modelo.append(1)
print('\nAjuste con MAE')
print('Error inicial -> ',ridge_error(modelo,x,y,reg_term,MAE))
linear_regression(modelo,x,y,2000,0.0001,reg_term,MAE_GRAD,False, True, False)
print('Error final -> ',ridge_error(modelo,x,y,reg_term,MAE))
print(modelo)

Ajuste lasso y ridge con conjunto de datos diabetes

---Ajuste lasso----
Ajuste con MSE
Error inicial ->  155162.06556399763
Error final ->  6428.112395020786
[0.010163943251526357, 0.0025245581279327257, 0.009771507878564196, 0.008396647760834012, 0.6456533737446366, -5.573579120435844e-05, 0.002199050003953748, 0.006284232170449195, 0.005815480502075072, 1.1074240660151344]

Ajuste con MAE
Error inicial ->  383.04527963800905
Error final ->  76.80651489729917
[0.009492760180995274, -0.003952941176472389, 0.0050464705882353736, -0.0021037918552037956, 0.44129185520360825, 0.002298552036200151, -0.0056276018099552065, 0.0061749004524881464, 0.005277526063348052, 1.1029153846153312]

---Ajuste ridge:---
Ajuste con MSE
Error inicial ->  155162.06556399763
Error final ->  5953.505224376461
[0.11036905129270543, 0.0031181404370293304, 0.07199985367175278, 0.23305021036658735, 0.3799188603844185, 0.22249727387790796, 0.06531802335444661, 0.011181011569029325, 0.011406803979208652, 1.0778729